## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Surat,IN,21.1667,72.8333,80.58,39,75,11.50,haze
1,1,Fairbanks,US,64.8378,-147.7164,-12.41,77,1,0.00,clear sky
2,2,Rikitea,PF,-23.1203,-134.9692,76.57,79,6,20.20,clear sky
3,3,Albany,US,42.6001,-73.9662,26.10,80,11,4.29,few clouds
4,4,Avarua,CK,-21.2078,-159.7750,77.05,65,40,10.36,scattered clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
max_tem = float(input("Enter the maximum temperature for the vacation! "))
min_tem = float(input("Enter the minmum temperature for the vacation! "))

Enter the maximum temperature for the vacation! 90
Enter the minmum temperature for the vacation! 70


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
vacation_cities = city_data_df.loc[(city_data_df["Max Temp"] <= max_tem) & (city_data_df["Max Temp"] >= min_tem)]
vacation_cities.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Surat,IN,21.1667,72.8333,80.58,39,75,11.50,haze
2,2,Rikitea,PF,-23.1203,-134.9692,76.57,79,6,20.20,clear sky
4,4,Avarua,CK,-21.2078,-159.7750,77.05,65,40,10.36,scattered clouds
5,5,Srandakan,ID,-7.9386,110.2506,83.89,68,100,8.41,overcast clouds
9,9,Butaritari,KI,3.0707,172.7902,81.09,80,80,9.10,light rain
10,10,Doha,QA,25.2867,51.5333,71.58,78,0,8.05,clear sky
12,12,Byron Bay,AU,-28.6500,153.6167,71.55,97,100,8.25,light rain
14,14,Cidreira,BR,-30.1811,-50.2056,70.21,67,16,14.70,few clouds
15,15,Kouroussa,GN,10.6500,-9.8833,70.66,28,99,1.77,overcast clouds
17,17,Hobart,AU,-42.8794,147.3294,83.01,47,20,4.00,few clouds


In [7]:
# 4a. Determine if there are any empty rows.
vacation_cities.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 250 entries, 0 to 686
Data columns (total 10 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City_ID              250 non-null    int64  
 1   City                 250 non-null    object 
 2   Country              249 non-null    object 
 3   Lat                  250 non-null    float64
 4   Lng                  250 non-null    float64
 5   Max Temp             250 non-null    float64
 6   Humidity             250 non-null    int64  
 7   Cloudiness           250 non-null    int64  
 8   Wind Speed           250 non-null    float64
 9   Current Description  250 non-null    object 
dtypes: float64(4), int64(3), object(3)
memory usage: 21.5+ KB


In [8]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = vacation_cities.dropna(how = "any")
clean_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Surat,IN,21.1667,72.8333,80.58,39,75,11.50,haze
2,2,Rikitea,PF,-23.1203,-134.9692,76.57,79,6,20.20,clear sky
4,4,Avarua,CK,-21.2078,-159.7750,77.05,65,40,10.36,scattered clouds
5,5,Srandakan,ID,-7.9386,110.2506,83.89,68,100,8.41,overcast clouds
9,9,Butaritari,KI,3.0707,172.7902,81.09,80,80,9.10,light rain
10,10,Doha,QA,25.2867,51.5333,71.58,78,0,8.05,clear sky
12,12,Byron Bay,AU,-28.6500,153.6167,71.55,97,100,8.25,light rain
14,14,Cidreira,BR,-30.1811,-50.2056,70.21,67,16,14.70,few clouds
15,15,Kouroussa,GN,10.6500,-9.8833,70.66,28,99,1.77,overcast clouds
17,17,Hobart,AU,-42.8794,147.3294,83.01,47,20,4.00,few clouds


In [9]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Surat,IN,80.58,haze,21.1667,72.8333,
2,Rikitea,PF,76.57,clear sky,-23.1203,-134.9692,
4,Avarua,CK,77.05,scattered clouds,-21.2078,-159.7750,
5,Srandakan,ID,83.89,overcast clouds,-7.9386,110.2506,
9,Butaritari,KI,81.09,light rain,3.0707,172.7902,
10,Doha,QA,71.58,clear sky,25.2867,51.5333,
12,Byron Bay,AU,71.55,light rain,-28.6500,153.6167,
14,Cidreira,BR,70.21,few clouds,-30.1811,-50.2056,
15,Kouroussa,GN,70.66,overcast clouds,10.6500,-9.8833,
17,Hobart,AU,83.01,few clouds,-42.8794,147.3294,


In [10]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():  
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()   

    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try: 
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except: 
        print("Skip the city")
        pass
        

Skip the city
Skip the city
Skip the city
Skip the city
Skip the city
Skip the city
Skip the city
Skip the city
Skip the city
Skip the city
Skip the city
Skip the city
Skip the city


In [11]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.dropna(how = "any")
clean_hotel_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 249 entries, 0 to 686
Data columns (total 7 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   City                 249 non-null    object 
 1   Country              249 non-null    object 
 2   Max Temp             249 non-null    float64
 3   Current Description  249 non-null    object 
 4   Lat                  249 non-null    float64
 5   Lng                  249 non-null    float64
 6   Hotel Name           249 non-null    object 
dtypes: float64(3), object(4)
memory usage: 15.6+ KB


In [12]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [19]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp}°F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [20]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level = 1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
# 11b. Display the figure
fig.add_layer(marker_layer)
fig

Figure(layout=FigureLayout(height='420px'))